## Load Mojo Kernels

In [1]:
import os, numpy, torch
from pathlib import Path
from max.torch import CustomOpLibrary

assert torch.cuda.is_available()
op_dir = os.path.abspath('operations')

## Simple `add_one` Operation

In [32]:
op_lib = CustomOpLibrary(Path(op_dir))
add_one = op_lib.my_add_constant[{"value": 1}]

In [12]:
import time

def torch_add_one(inputs):
    return inputs + 1

def mojo_add_one(inputs):
    outputs = torch.zeros_like(inputs)
    add_one(outputs, inputs)
    return outputs

for device in ["cpu", "cuda"]:
    for op in [torch_add_one, mojo_add_one]:
        x = torch.zeros(1024, device=device)
        x = op(x) # warm-up
        start = time.perf_counter()
        for _ in range(1000):
            x = op(x)
        end = time.perf_counter()
        print(op.__name__, device, x, end - start)

torch_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.0034156449837610126
mojo_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.13251558190677315
torch_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.0074024859350174665
mojo_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.11501298600342125


## Different MatMul Algorithms

In [2]:
from max.driver import Accelerator, accelerator_count, Tensor
import torch
M, K, N = 8, 8, 16 # debug
M, K, N = 4096, 6144, 2048 # real-world scale
device = Accelerator()
torch_A = torch.randn(M, K)
torch_B = torch.randn(K, N)
torch_result = (torch_A @ torch_B).numpy()
A = Tensor.from_numpy(torch_A.numpy()).to(device)
B = Tensor.from_numpy(torch_B.numpy()).to(device)

Build and test executing the CUDA graph for our MatMul kernel:

In [3]:
from max.graph import Graph, TensorType, DeviceRef, ops
def build_graph(session, algorithm):
    print('building cuda graph for', algorithm)
    with Graph("matmul_graph",
               input_types=[
                   TensorType(dtype=A.dtype, shape=A.shape, device=DeviceRef.from_device(device)),
                   TensorType(dtype=B.dtype, shape=B.shape, device=DeviceRef.from_device(device))
               ],
               custom_extensions=[Path(op_dir)]) as graph:
        A_value, B_value = graph.inputs
        output = ops.custom(
            name="my_matmul",
            device=DeviceRef.from_device(device),
            values=[A_value, B_value],
            out_types=[
                TensorType(dtype=A.dtype, shape=[
                        A_value.tensor.shape[0], B_value.tensor.shape[1]
                    ], device=DeviceRef.from_device(device))
            ],
            parameters={"algorithm": algorithm},
        )
        graph.output(output[0].tensor)
    print('loading cuda graph...')
    return session.load(graph) # compile the graph

from max.engine import InferenceSession
session = InferenceSession(devices=[device])
graph =  build_graph(session, "tiled_register")
mojo_result = graph.execute(A, B)[0].to_numpy()
print("test run:\n", mojo_result, end="\n\n")
print("reference:\n", torch_result)

building cuda graph for tiled_register
loading cuda graph...
test run:
 [[  12.389246   -153.94875      79.36485    ...  130.34673
    90.88082     -96.9761    ]
 [ -46.51677    -119.79422     -66.22736    ...   13.707309
    -0.17698337   22.379784  ]
 [  28.513195    -56.576187    -28.25098    ...   39.71024
   -47.64227     118.266235  ]
 ...
 [  74.7953       41.05835      31.518501   ...   -8.026637
   151.28687     -42.16699   ]
 [  24.267195    -91.36723     -90.63792    ... -107.86867
   -19.431274     65.96532   ]
 [  74.05394      83.50616      78.99106    ... -101.248795
   -99.03192     -11.61975   ]]

reference:
 [[  12.389119   -153.9488       79.36478    ...  130.34688
    90.880775    -96.976135  ]
 [ -46.516716   -119.79383     -66.22738    ...   13.707359
    -0.17699242   22.37973   ]
 [  28.51316     -56.576237    -28.250923   ...   39.710224
   -47.642204    118.266136  ]
 ...
 [  74.79533      41.058327     31.51856    ...   -8.026703
   151.28703     -42.166946  

Verify kernel results:

In [4]:
assert numpy.allclose(mojo_result, torch_result, rtol=0, atol=0.005)
for row in range(torch_result.shape[0]):
    if numpy.allclose(torch_result[row], mojo_result[row], rtol=0, atol=0.005): continue
    print('mismatch row:', row)

The `tiled_register` kernel can be visualized using the [matmul_visualization.mojo](./matmul_visualization.mojo) and `matmul_visualization_gui/*` scripts.

Here is a screenshot:
![](./GEMM_visualization.png)

Run a complete benchmark for different algorithms:

In [7]:
import time
for algo in ["naive", "coalescing", "tiled", "tiled_register"]:
    graph =  build_graph(session, algo)
    record = dict(torch=0, mojo=0)
    sampels = 5
    for _ in range(sampels):
        torch_A = torch.randn(M, K).to('cuda:0')
        torch_B = torch.randn(K, N).to('cuda:0')
        A = Tensor.from_numpy(torch_A.cpu().numpy()).to(device)
        B = Tensor.from_numpy(torch_B.cpu().numpy()).to(device)
        # torch
        torch.cuda.synchronize()
        begin = time.perf_counter()
        torch_result = torch_A @ torch_B
        torch.cuda.synchronize()
        record['torch'] += (time.perf_counter() - begin) / sampels
        # mojo
        torch.cuda.synchronize()
        begin = time.perf_counter()
        mojo_result = graph.execute(A, B)
        torch.cuda.synchronize()
        record['mojo'] += (time.perf_counter() - begin) / sampels
        assert numpy.allclose(mojo_result[0].to_numpy(), torch_result.cpu().numpy(), rtol=0, atol=0.005)
    print(algo, record)

building cuda graph for naive
loading cuda graph...
naive {'torch': 0.013650156208314002, 'mojo': 0.5180980710312724}
building cuda graph for coalescing
loading cuda graph...
coalescing {'torch': 0.013412875006906688, 'mojo': 0.134530279180035}
building cuda graph for tiled
loading cuda graph...
tiled {'torch': 0.014552333392202854, 'mojo': 0.12279981463216244}
building cuda graph for tiled_register
loading cuda graph...
tiled_register {'torch': 0.013400612189434467, 'mojo': 0.2502177455928177}


## Reference
[1] https://github.com/modular/modular/blob/main/examples/custom_ops/kernels/matrix_multiplication.mojo

[2] https://docs.modular.com/max/tutorials/custom-ops-matmul